In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
# from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:
ANSATZs=[]
ENCODERs=[]
pairs=[]
results={}
for i in glob('../BSE_RUD_AL_test/5qubit/3_AL/*/*results.json'):
    pair = os.path.basename(i).replace('_results.json','')
    pairs.append(pair)
    an, en = pair.split('_')
    ANSATZs.append(an)
    ENCODERs.append(en)
    


ANSATZs=set(ANSATZs)
ENCODERs=set(ENCODERs)

In [ ]:


# device='qiskit.aer'
# device_backend = FakeCairoV2()
# backend = AerSimulator.from_backend(device_backend)
# device = qml.device(device, wires=num_qubits, backend=backend, shots=shots)


device='qulacs.simulator'
num_qubits=5
device = qml.device(device, wires=num_qubits)


In [ ]:
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = initial_parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(num_qubits))
        variational(params, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))

In [ ]:
def get_depth(enc,var,rud,features,parameters):
    circuit_gates=0
    circuit_depth=0
    for i in range(rud):
        params = initial_parameters.reshape(rud,-1)[i]
        
        encoder_specs=qml.specs(qml.QNode(encoder,device,expansion_strategy='device'))(features, wires=range(num_qubits))['resources']
        var_specs=qml.specs(qml.QNode(variational,device,expansion_strategy='device'))(params, wires=range(num_qubits))['resources']

        circuit_gates+=var_specs.num_gates + encoder_specs.num_gates
        circuit_depth+=var_specs.depth + encoder_specs.depth
    return circuit_gates,circuit_depth

In [ ]:


# draw_encoder = qml.draw_mpl(qml.QNode(ENCODER_LIST['IQP'],device),level=3)
# draw_encoder(np.random.rand(num_qubits), range(num_qubits))

In [ ]:
results={}
flat_results=[]
for d in [3, 5]:
    results[d]={}
    for a in ['AL','RUD']:
        results[d][a]={}
        if a=='AL':
            LAYERS=d
            re_upload_depth=1
        else:
            LAYERS=1
            re_upload_depth=d
            
        for p in pairs:
            results[d][a][p]={}
            depth_path = os.path.join("../BSE_RUD_AL_test/5qubit/",'_'.join((str(d),a)),p)
            if os.path.exists(depth_path):
                with open(os.path.join(depth_path,f'{p}_results.json'),'r') as f:
                    results_dict=json.load(f)
                
                
            ENCODER, ANSATZ = p.split('_')
            # print(ENCODER, ANSATZ)
            encoder = ENCODER_LIST[ENCODER]
            variational=ANSATZ_LIST[ANSATZ]
            variational.layers = LAYERS
            variational.set_wires(range(num_qubits))
     

            num_params = variational.num_params * re_upload_depth
            generator = np.random.default_rng(12958234)
            initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
            feat = generator.uniform(-1, 1, num_qubits)
            
            qnode = qml.QNode(circuit, device)        
            circuit_specs=qml.specs(qnode)(feat,initial_parameters)['resources']

            circuit_gates,circuit_depth= get_depth(encoder,variational,re_upload_depth,feat,initial_parameters)
            # print(circuit_specs.num_gates,circuit_specs.depth)
            # print(circuit_gates,circuit_depth)
            
            # print(f"{p} AL_{LAYERS} RUD_{re_upload_depth}\n",circuit_specs)
            # qml.draw_mpl(qnode,level=1)(feat,initial_parameters)
            # plt.show()
            # print(variational.layers)
            # print()

            results[d][a][p]={'MAE_train':results_dict['MAE_train'],'MAE_test':results_dict['MAE_test'],'num_gates':circuit_gates,'depth':circuit_depth}
            if a=='AL':
                flat_results.append((d,1,p,results_dict['MAE_train'],results_dict['MAE_test'],circuit_gates,circuit_depth))
            else:
                flat_results.append((1,d,p,results_dict['MAE_train'],results_dict['MAE_test'],circuit_gates,circuit_depth))
            

In [ ]:
for p in pairs:
    with open(f'../BSE-calc-test/5qubit/{p}/{p}_results.json','r') as f:
        og_results=json.load(f)
        
    ENCODER, ANSATZ = p.split('_')
    # print(ENCODER, ANSATZ)
    encoder = ENCODER_LIST[ENCODER]
    variational=ANSATZ_LIST[ANSATZ]

    LAYERS=1
    re_upload_depth=1
    variational.layers = LAYERS
    variational.set_wires(range(num_qubits))
    

    num_params = variational.num_params * re_upload_depth
    generator = np.random.default_rng(12958234)
    initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
    feat = generator.uniform(-1, 1, num_qubits)
    
    qnode = qml.QNode(circuit, device)   

    # print(f"{p} AL_{LAYERS} RUD_{re_upload_depth}\n",circuit_specs)
    # qml.draw_mpl(qnode,level=1)(feat,initial_parameters)
    # plt.show()
    
    circuit_specs=qml.specs(qnode)(feat,initial_parameters)['resources']

    circuit_gates,circuit_depth= get_depth(encoder,variational,1,feat,initial_parameters)
    flat_results.append((1,1,p,og_results['MAE_train'],og_results['MAE_test'],circuit_gates,circuit_depth))

In [ ]:
df = pd.DataFrame(flat_results,columns=['AL','RUD','pairs','MAE Train','MAE Test','num_gates','depth'])

In [ ]:
sns.histplot(df['depth'])

In [ ]:
len(ENCODERs)//2

In [ ]:
df['diff']=(df['MAE Train']-df['MAE Test']).abs()

In [ ]:
df[(df['MAE Train']>0.9)&(df['MAE Train']>0.9)].sort_values(by=['MAE Train','MAE Test'])

In [ ]:
median_Train,median_Test=np.median(df[['MAE Train',"MAE Test"]].values,axis=0)

In [ ]:
palette=sns.color_palette('Paired',4)

In [ ]:
palette

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 16
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

fig,ax=plt.subplots(3,len(pairs)//3,sharey=True,sharex=True,figsize=(25,10))


for idxe,p in enumerate(pairs):
    
        pairdf=df[df['pairs']==p].sort_values(by=["num_gates"])
        min=pairdf.sort_values(by=['MAE Train',"MAE Test"]).iloc[0]
        ax[idxe%3,idxe%5].plot(pairdf['num_gates'],pairdf['MAE Train'],label=f'Train'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')',color=palette[1])
        ax[idxe%3,idxe%5].plot(pairdf['num_gates'],pairdf['MAE Test'],label=f'Test'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')',color=palette[3])
        ax[idxe%3,idxe%5].scatter(min['num_gates'],min['MAE Train'],color=palette[1])
        ax[idxe%3,idxe%5].scatter(min['num_gates'],min['MAE Test'],color=palette[3])
        ax[idxe%3,idxe%5].hlines(median_Train,-10,500,color='r',linestyles=':',label="Median Train MAE")
        ax[idxe%3,idxe%5].hlines(median_Test,-10,500,color='b',linestyles=':',label="Median Test MAE")
        ax[idxe%3,idxe%5].legend(ax[idxe%3,idxe%5].get_legend_handles_labels()[0][:-2],ax[idxe%3,idxe%5].get_legend_handles_labels()[1][:-2],loc=1)
        ax[idxe%3,idxe%5].set_ylabel('MAE (kcal/mol)')
        ax[idxe%3,idxe%5].set_xlim(0,450)
        ax[idxe%3,idxe%5].set_title(p)

        if (idxe%3)==2:
            ax[idxe%3,idxe%5].set_xlabel('Number of Gates')

fig.legend(ax[-1,-1].get_legend_handles_labels()[0][-2:],ax[-1,-1].get_legend_handles_labels()[1][-2:], loc='right', bbox_to_anchor=(1.1, 0.9))        
plt.tight_layout()        
plt.savefig('BSE5_numgates_vs_MAE.png',dpi=300)
plt.show()

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 16
BIGGER_SIZE = 12

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

fig,ax=plt.subplots(3,len(pairs)//3,sharey=True,sharex=True,figsize=(25,10))


for idxe,p in enumerate(pairs):
    
        pairdf=df[df['pairs']==p].sort_values(by=["depth"])
        min=pairdf.sort_values(by=['MAE Train',"MAE Test"]).iloc[0]
        ax[idxe%3,idxe%5].plot(pairdf['depth'],pairdf['MAE Train'],label=f'Train'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')',color=palette[1])
        ax[idxe%3,idxe%5].plot(pairdf['depth'],pairdf['MAE Test'],label=f'Test'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')',color=palette[3])
        ax[idxe%3,idxe%5].scatter(min['depth'],min['MAE Train'],color=palette[1])
        ax[idxe%3,idxe%5].scatter(min['depth'],min['MAE Test'],color=palette[3])
        ax[idxe%3,idxe%5].hlines(median_Train,-10,500,color='r',linestyles=':',label="Median Train MAE")
        ax[idxe%3,idxe%5].hlines(median_Test,-10,500,color='b',linestyles=':',label="Median Test MAE")
        ax[idxe%3,idxe%5].legend(ax[idxe%3,idxe%5].get_legend_handles_labels()[0][:-2],ax[idxe%3,idxe%5].get_legend_handles_labels()[1][:-2],loc=1)
        ax[idxe%3,idxe%5].set_ylabel('MAE (kcal/mol)')
        ax[idxe%3,idxe%5].set_xlim(0,200)
        ax[idxe%3,idxe%5].set_title(p)

        if (idxe%3)==2:
            ax[idxe%3,idxe%5].set_xlabel('Circuit Depth')

fig.legend(ax[-1,-1].get_legend_handles_labels()[0][-2:],ax[-1,-1].get_legend_handles_labels()[1][-2:], loc='right', bbox_to_anchor=(1.1, 0.9))                
plt.tight_layout()        
plt.savefig('BSE5_circuitdepth_vs_MAE.png',dpi=300)
plt.show()

In [ ]:
# SMALL_SIZE = 8
# MEDIUM_SIZE = 14
# BIGGER_SIZE = 12

# plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
# plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
# plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
# plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
# plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
# plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title

# fig,ax=plt.subplots(2,len(ENCODERs)//2,sharey=True,sharex=True,figsize=(25,10))

# for idxa,a in enumerate(ANSATZs):
#     for idxe,e in enumerate(ENCODERs):
#         if '_'.join((a,e)) in pairs:
#             pairdf=df[df['pairs']=='_'.join((a,e))].sort_values(by=["num_gates"])
#             min=pairdf.sort_values(by=['MAE Train',"MAE Test"]).iloc[0]
#             ax[idxe%2,idxe%5].plot(pairdf['num_gates'],pairdf['MAE Train'],label=f'Train {a}'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')')
#             ax[idxe%2,idxe%5].plot(pairdf['num_gates'],pairdf['MAE Test'],label=f'Test {a}'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')')
#             ax[idxe%2,idxe%5].scatter(min['num_gates'],min['MAE Train'])
#             ax[idxe%2,idxe%5].scatter(min['num_gates'],min['MAE Test'])
#             ax[idxe%2,idxe%5].hlines(median_Train,-10,500,color='r',linestyles=':',label="Median Train MAE")
#             ax[idxe%2,idxe%5].hlines(median_Test,-10,500,color='b',linestyles=':',label="Median Test MAE")
#             ax[idxe%2,idxe%5].legend(loc=2)
#             ax[idxe%2,idxe%5].set_ylabel(e)
#             ax[idxe%2,idxe%5].set_xlim(0,450)

#             if (idxe%2)==1:
#                 ax[idxe%2,idxe%5].set_xlabel('Number of Gates')
    
        
# plt.tight_layout()        
# plt.show()

In [ ]:
# fig,ax=plt.subplots(2,len(ENCODERs)//2,sharey=True,sharex=True,figsize=(25,10))

# for idxa,a in enumerate(ANSATZs):
#     for idxe,e in enumerate(ENCODERs):
#         if '_'.join((a,e)) in pairs:
#             pairdf=df[df['pairs']=='_'.join((a,e))].sort_values(by=["depth"])
#             min=pairdf.sort_values(by=['MAE Train',"MAE Test"]).iloc[0]
#             ax[idxe%2,idxe%5].plot(pairdf['depth'],pairdf['MAE Train'],label=f'Train {a}'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')')
#             ax[idxe%2,idxe%5].plot(pairdf['depth'],pairdf['MAE Test'],label=f'Test {a}'+" ("+'_'.join((f"RUD{min['RUD']}",f"AL{min['AL']}"))+f')')
#             ax[idxe%2,idxe%5].scatter(min['depth'],min['MAE Train'])
#             ax[idxe%2,idxe%5].scatter(min['depth'],min['MAE Test'])
#             ax[idxe%2,idxe%5].hlines(median_Train,-10,300,color='r',linestyles=':',label="Median Train MAE")
#             ax[idxe%2,idxe%5].hlines(median_Test,-10,300,color='b',linestyles=':',label="Median Test MAE")
#             ax[idxe%2,idxe%5].legend(loc=2)
#             ax[idxe%2,idxe%5].set_ylabel(e)
#             ax[idxe%2,idxe%5].set_xlim(0,200)

#             if (idxe%2)==1:
#                 ax[idxe%2,idxe%5].set_xlabel('Circuit Depth')
    
        
# plt.tight_layout()        
# plt.show()

In [ ]:
# for p in pairs:
#     sns.lineplot(df[df['pairs']==p].melt(id_vars=['num_gates','depth'],value_vars=['MAE Train','MAE Test']),x='depth',y='value',hue='variable')
#     plt.show()

In [ ]:
layerdf=df.melt(id_vars=['AL','RUD'],value_vars=['MAE Train','MAE Test'])

In [ ]:
fig,ax=plt.subplots(1,2,sharey=True,figsize=(10,5))
sns.boxplot(data=layerdf,y='value',hue='variable',x='AL',ax=ax[0],palette=sns.color_palette('Paired',2))
sns.boxplot(data=layerdf,y='value',hue='variable',x='RUD',ax=ax[1],palette=sns.color_palette('Paired',2))
# ax[0].hlines(median_Train,-10,500,color='r',linestyles=':',label="Median Train MAE")
# ax[0].hlines(median_Test,-10,500,color='b',linestyles=':',label="Median Test MAE")
# ax[1].hlines(median_Train,-10,500,color='r',linestyles=':',label="Median Train MAE")
# ax[1].hlines(median_Test,-10,500,color='b',linestyles=':',label="Median Test MAE")
ax[0].legend().set_visible(False)
ax[1].legend().set_visible(False)
ax[0].set_xlim(-0.5,2.5)
ax[1].set_xlim(-0.5,2.5)
ax[0].set_ylim(15,45)
ax[1].set_ylim(15,45)
ax[0].set_ylabel('MAE (kcal/mol)')
ax[0].set_xlabel('Ansatz Layers')
ax[1].set_xlabel('Re-upload Depth')
fig.legend(ax[1].get_legend_handles_labels()[0],ax[1].get_legend_handles_labels()[1], loc='right', bbox_to_anchor=(1.2, 0.9))                
plt.tight_layout()
plt.savefig('BSE5_MAE_vs_layers.png',dpi=300)
plt.show()